Em dezembro, a curva de juros apresentou fechamento em relação ao mês anterior em todos os vértices. No último mês do ano, o cenário político doméstico seguiu como o principal direcionador do mercado. Além disso, outros indicadores econômicos deram suporte ao movimento, como o IPCA-15 de dezembro, que apresentou valores abaixo do consenso de mercado. As taxas dos títulos soberanos prefixados ficaram abaixo dos 13% e o juro real se mantém próximo dos 6%.

A Poupança apresentou captação líquida de R$ 6,3 bilhões, contrariando a tendência negativa dos últimos meses – possível reflexo do pagamento de 13º salário. Os títulos públicos tiveram rentabilidade majoritariamente positiva devido, principalmente, ao fechamento da curva de juros. O Tesouro Direto, por sua vez, apresentou captação líquida de R$ 805 milhões em novembro (último dado disponível), com maior participação dos títulos pós-fixados. Os estoques de emissões bancárias e crédito privado continuaram a subir na maioria dos títulos, indicando mercado aquecido e rentabilidades mais altas.

Para o primeiro e próximos meses meses de 2023, as atenções nos mercados devem focar na política fiscal do novo governo. Além disso, o mercado global e riscos de recessão permanecem no radar. Nosso time de Economia espera a manutenção da taxa Selic no patamar de 13,75%.

In [1]:
pip install tesouro-direto-br

In [12]:
import getpass
import tesouro_direto_br as tesouro_direto

#Criar uma carteira vazia:
carteira = tesouro_direto.Carteira(tesouro_direto.Titulo())

#Adicionar titulos publicos:
titulo1 = tesouro_direto.Titulo("Tesouro IPCA+", "2026-08-15", "2021-07-08", 33.65)
titulo2 = tesouro_direto.Titulo("Tesouro Selic", "2025-03-01", "2021-07-08", 50)

carteira.add(titulo1)
carteira.add(titulo2)

In [5]:
USER = getpass.getuser().lower()
PASS = getpass.getpass("Senha de rede: ")
SERVIDOR = "SERVIDOR"
PORTA = "PORTA"

proxies = {
          "http":f"http://{USER}:{PASS}@{SERVIDOR}:{PORTA}",
          "https":f"http://{USER}:{PASS}@{SERVIDOR}:{PORTA}",
}

carteira_tesouro_direto = tesouro_direto.calcula_retorno_carteira(carteira, proxies=proxies)

Senha de rede: ··········


InvalidURL: Failed to parse: http://root:@SERVIDOR:PORTA

In [6]:
retorno_carteira, retorno_titulos_acumulado = tesouro_direto.calcula_mtm_carteira(carteira_tesouro_direto)

ano, mes, dia = str(retorno_carteira.index[0]).split('-')
dia = dia.split(" ")[0]
retorno_periodo = retorno_carteira.iloc[-1]*100
print(f"Retorno da Carteira é de {round(retorno_periodo,2)}% desde {dia}/{mes}/{ano}")

AttributeError: module 'tesouro_direto_br' has no attribute 'calcula_mtm_carteira'

In [7]:
rentalibidade_periodo = 100*carteira_tesouro_direto['Rentabilidade Acumulada'].iloc[-1]
print(f"Rentabilidade do período é de {round(rentalibidade_periodo, 3)}%")

NameError: name 'carteira_tesouro_direto' is not defined

In [8]:
import matplotlib.pyplot as plt
import comparar_fundos_br as comp

cdi, cdi_acumulado = comp.get_benchmark(str(carteira_tesouro_direto.index[0]).split(" ")[0],
                                        str(carteira_tesouro_direto.index[-1]).split(" ")[0],
                                        benchmark = "cdi", proxy=proxies)
retorno_cdi = (cdi_acumulado-1)*100

plt.figure(figsize=(17,5))
plt.title("Rentabilidade da Carteira")
plt.plot(carteira_tesouro_direto[["Rentabilidade Acumulada"]]*100)
plt.plot(retorno_cdi, color="red", linestyle="--", lw=1)
plt.legend(["Carteira", "CDI"], frameon=False, loc="center right")
plt.ylabel('%',rotation=0,labelpad=-10,loc="top")
plt.grid(axis="x")
plt.show()

ModuleNotFoundError: No module named 'comparar_fundos_br'

In [9]:
nom = tesouro_direto.nomeclatura_titulos()
for tpf in carteira.titulos:
    tipo_titulo = tpf["Tipo"]
    investimento = tpf["Quantidade"]
    data_investimento = tpf["Data Investimento"]
    vencimento = tpf["Vencimento"]
    venc = vencimento.split("-")[0]
    nome_titulo = [nom[tipo_titulo].upper() + "_" + venc + "_" + data_investimento]
    print(nome_titulo[0])
    mtm_titulo = carteira_tesouro_direto[nome_titulo].dropna()
    custos, detalhamento_custos = get_custos( mtm_titulo, custo_b3=True)
    print()

KeyError: 'Quantidade'

In [10]:
movimentacao_tpf = movimentacoes_titulos_publicos("venda", proxies=proxies)

#Maiores Movimentações nos últimos 10 dias:
maiores_movimentacoes = movimentacao_pivot.iloc[-10:].T.dropna()
maiores_movimentacoes["Soma"] = maiores_movimentacoes.sum(axis=1)
maiores_movimentacoes = maiores_movimentacoes.sort_values("Soma", ascending=False)
print(maiores_movimentacoes)

#destaque para os 3 maiores movimentos de venda
tres_maiores = maiores_movimentacoes.index[:3].tolist()
maiores = maiores_movimentacoes.drop(["Soma"], axis=1).T

fig = plt.gcf()
fig.set_size_inches(15, 7, forward = False)
plt.rcParams.update({'font.size': 22})

for tit in maiores.drop(tres_maiores, axis=1).columns:
    plt.plot(maiores[tit], color="gray", alpha=0.35, label='')
for tit in maiores[tres_maiores].columns:
    plt.plot(maiores[tit], lw=3, label=tit)
plt.legend(frameon=False, bbox_to_anchor=(0.95, 1.))
plt.suptitle("Movimentações de Venda de Títulos Públicos")
plt.box(False)
plt.grid(axis="y")
plt.ylabel('Quantidade',rotation=0,labelpad=-30,loc="top")
plt.xticks(rotation=15)
plt.show()

NameError: name 'movimentacoes_titulos_publicos' is not defined

#Oferta

In [13]:
titulos_ofertados = tesouro_direto.busca_tesouro_direto(tipo="taxa", proxies=proxies).reset_index()

excluir = ["Juros Semestrais", "Renda+", "Educa+"]
titulos_ofertados_filtrado = titulos_ofertados[(titulos_ofertados["Data Base"]>="2016-02-17") &
                                   (titulos_ofertados["Data Base"]<"2017-01-01") &
                                   (~titulos_ofertados["Tipo Titulo"].str.contains("&".join(excluir)))].set_index(["Tipo Titulo",
                                                                                    "Data Vencimento"])
titulos_ofertados_filtrado.tail()

InvalidURL: Failed to parse: http://root:@SERVIDOR:PORTA